In [30]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib as plt
import itertools as it
from analysis_utils import *

In [31]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [32]:
sns.set_style('white')

# Setup

In [29]:
EXPERIMENT = 8

if EXPERIMENT == (5, 7, 8):
    print(f'Use exp{EXPERIMENT}.ipynb')
    exit()

os.makedirs(f'stats/{EXPERIMENT}', exist_ok=True)
def write_tex(name, tex):
    file = f"stats/{EXPERIMENT}/{name}.tex"
    with open(file, "w+") as f:
        f.write(str(tex) + r"\unskip")
    print(f'wrote "{tex}" to "{file}"')

data = get_data(EXPERIMENT)
pdf = data['participants'].rename(columns={'wid': 'pid'}).set_index('pid').copy()
full_pdf = pdf.copy()
mdf = data['trials'].rename(columns={'wid': 'pid'}).set_index('pid').copy()
mdf.trial_time /= 1000
mdf['clicked'] = mdf.n_clicks > 0

full_pdf = pdf.copy()

if EXPERIMENT == 3:
    pdf.query('completed_1', inplace=True)
else:
    pdf.query('completed', inplace=True)
    

if EXPERIMENT == 6:
    pdf.rename(columns={'with_info': 'information', 'with_reward': 'reward'}, inplace=True)
    fb_namer = {
        (0, 0): 'none',
        (1, 0): 'info_only',
        (0, 1): 'reward_only',
        (1, 1): 'both'
    }
    pdf['feedback'] = pdf.apply(lambda row: fb_namer[row.information, row.reward], axis=1)
    mdf['information'] = pdf.information
    mdf['reward'] = pdf.reward
    fb_order = list(fb_namer.values())
elif EXPERIMENT == 7:
    pdf.rename(columns={'constantTest': 'constant_test'}, inplace=True)
    mdf['constant_test'] = pdf.constant_test
    pdf['feedback'] = 'meta'
    mdf['feedback'] = 'meta'
else:
    fb_order = [fb for fb in ['none', 'action', 'meta'] if fb in set(pdf.feedback)]

AttributeError: 'DataFrame' object has no attribute 'feedback'

In [ ]:
def expected_score(row):
    path = map(int, row.path[1:])
    # assert sum(row.state_rewards[p] for p in path) - 3 * len(row.clicks) == row.score
    clicks = set(row.clicks)
    cost = {'test': 3, 'training': 1}[row.block]
    return sum(row.state_rewards[p] if p in clicks else 0 for p in path) - cost * len(clicks)

# mdf['expected_score'] = mdf.apply(expected_score, axis=1)
# pdf['expected_test'] = mdf.query('block == "test"').groupby('pid').expected_score.mean()
    
# row.state_rewards

## Demographics

In [16]:
pids = list(pdf.index.unique())
sdf = data['survey'].rename(columns={'wid': 'pid'}).query('pid == @pids').copy()
if not isinstance(sdf.responses.iloc[0], dict):
    sdf = sdf.loc[~sdf.responses.isna()]
    sdf.responses = sdf.responses.apply(ast.literal_eval)

if EXPERIMENT == 3:
    demo = sdf.loc[sdf.responses.apply(len) == 3].set_index('pid').responses
    age = demo.apply(get('Q1'))
    gender = demo.apply(get('Q2'))
else:
    demo = sdf.loc[sdf.responses.apply(len) == 2].set_index('pid').responses
    age = demo.apply(get('Q0'))
    gender = demo.apply(get('Q1'))
    
gender = gender.str.lower()
age = age.apply(excepts(ValueError, int, lambda _: None))
age.loc[age < 18] = None

write_tex('mean-age', f'{age.mean():.2f}')
write_tex('min-age', str(age.min()))
write_tex('max-age', str(age.max()))

regularize = {
    'man': 'male',
    'woman': 'female',
    'f': 'female',
    'm': 'male',
}
gender = gender.apply(lambda x: regularize.get(x.strip(), x))
write_tex("N-female", str(gender.value_counts()['female']))

write_tex("N-total", f"${len(pdf)}$")
for fb, n in pdf.feedback.value_counts().items():
    write_tex(f"N-{fb}", f"${n}$")
    
write_tex('mean-bonus', f'\${pdf.bonus.mean():.2f}')

wrote "34.78" to "stats/7/mean-age.tex"
wrote "18.0" to "stats/7/min-age.tex"
wrote "71.0" to "stats/7/max-age.tex"
wrote "43" to "stats/7/N-female.tex"
wrote "$102$" to "stats/7/N-total.tex"
wrote "$102$" to "stats/7/N-meta.tex"
wrote "\$1.43" to "stats/7/mean-bonus.tex"


In [17]:
completed = list(pdf.index)
mdf.reset_index(inplace=True)
mdf.query('pid == @completed', inplace=True)
mdf.set_index('pid', inplace=True)

pdf.feedback = pd.Categorical(pdf.feedback, fb_order, ordered=True)    
mdf['feedback'] = pdf.feedback
block_mean = mdf.groupby(['block', 'pid']).score.mean()
for b in ['training', 'test']:
    pdf[b] = block_mean[b]

In [18]:
if EXPERIMENT == 3:
    dropped = pdf.query('not completed').feedback.value_counts()
    rate = dropped / pdf.feedback.value_counts()
    for fb in fb_order:
        write_tex(f'N-drop-{fb}', dropped[fb])
        write_tex(f'drop-rate-{fb}', f'${rate[fb]*100:.1f}\%$')
    write_tex('return-rate', f'${pdf.completed.mean()*100:.1f}\%$')
    write_tex('return-N', f'${pdf.completed.sum()}$')
    pdf.query('completed', inplace=True)
    
pd.Series(pdf.index).to_csv(f'pids/{EXPERIMENT}.csv', index=False, header=False)

In [19]:
small_leaves = {3,4,7,8,11,12}
transfer_leaves = {5, 6, 7, 9, 10, 11, 16, 17, 18, 20, 21, 22, 27, 28, 29, 31, 32, 33}

def clicked_leaf_first(row):
    if not row.clicks:
        return False
    first = row.clicks[0]
    transfer =  EXPERIMENT in (2, 3) and row.block == "test"
    leaves = transfer_leaves if transfer else small_leaves
    return first in leaves

mdf['backward'] = mdf.apply(clicked_leaf_first, axis=1)
pdf['first_backward'] = mdf.query('block == "train" and trial_index == 0').backward

def n_leaf(row):
    if not row.clicks:
        return False
    transfer =  EXPERIMENT in (2, 3) and row.block == "test"
    leaves = transfer_leaves if transfer else small_leaves
    return sum(click in leaves for click in row.clicks)

mdf['n_leaf'] = mdf.apply(n_leaf, axis=1).astype(int)

In [22]:
if EXPERIMENT not in (1, 6, 7):  # run before this was a problem
    write_tex('N-drop-previous', sum(pdf.previously_participated != "No"))
    print(pdf.previously_participated.value_counts())
    pdf = pdf.query('previously_participated == "No"')
    # pdf['first_backward'] = mdf.query('trial_index == 0').backward
    # pdf = pdf.query('not first_backward')
    mdf = mdf.loc[pdf.index]

In [23]:
n_drop = len(full_pdf) - len(pdf)
write_tex('N-drop', f'{n_drop}')
write_tex('percent-drop', f'{100 * n_drop / len(full_pdf):.1f}\%')

wrote "9" to "stats/7/N-drop.tex"
wrote "8.1\%" to "stats/7/percent-drop.tex"


In [24]:
pd.Series(pdf.index).to_csv(f'../data/{EXPERIMENT}/included_pids.csv', index=False, header=True)

## Setup Plotting 

In [ ]:
figure = Figures(path=f'figs/{EXPERIMENT}', formats=['pdf']).plot

sns.set_style('whitegrid')
blue, orange = sns.color_palette('tab10')[:2]
gray = (0.5,)*3
red = (1, 0.2, 0.3)
yellow = (0.9, 0.85, 0)
palette = {
    'none': gray,
    'action': blue,
    'meta': orange,
    'info_only': red,
    'reward_only': yellow,
    'both': orange,
}

palette = {
    'none': gray,
    'action': blue,
    'meta': orange,
    'info_only': red,
    'reward_only': yellow,
    'both': orange,
}

nice_names = {
    'meta': 'Metacognitive',
    'action': 'Action',
    'none': 'None',
    'feedback': 'Feedback',
    'info_only': 'Information\nOnly',
    'reward_only': 'Delay Penalty\nOnly',
    'both': 'Information &\nDelay Penalty',
    'score': 'Average Score',
    'backward': 'Proportion Planning Backward'
}

def reformat_labels(ax=None):
    ax = ax or plt.gca()
    labels = [t.get_text() for t in ax.get_xticklabels()]
    new_labels = [nice_names.get(lab, lab) for lab in labels]
    ax.set_xticklabels(new_labels)
    
def reformat_legend(ax=None):
    if ax is None:
        ax = plt.gca()
    handles, labels = ax.get_legend_handles_labels()
    print([nice_names.get(l, l).replace('\n', ' ') for l in labels])
    ax.legend(handles=handles, labels=[nice_names.get(l, l).replace('\n', ' ') 
                                       for l in labels])
    
def plot_block_changes():
    block_changes = mdf.loc[1].block.apply(Labeler()).diff().reset_index().query('block == 1').index
    for t in block_changes:
        plt.axvline(t-0.5, c='k', ls='--')

from datetime import datetime
# os.makedirs(f'stats/{EXPERIMENT}/', exist_ok=True)
def result_file(name, ext='tex'):
    file = f'stats/{EXPERIMENT}-{name}.{ext}'
#     with open(file, 'w+') as f:
#         timestamp = datetime.now().strftime('Created on %m/%d/%y at %H:%M:%S\n\n')
#         f.write(timestamp)
    return file

# Test score

In [ ]:
def write_kruskal(name):
    out = %R kruskal.test($name ~ feedback, data=rdf)
    out = dict(out.items())
    df = out["parameter"][0]
    p = pval(out["p.value"][0])
    stat = out["statistic"][0]
    write_tex(f'{name}-kruskal', rf'$H = {stat:.3f}, {p}$')

def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    pooled_std =  np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)
    return (np.mean(x) - np.mean(y)) / pooled_std

import re
def write_independence_test(y, a, b, alternative='two.sided'):
    # oddly, feedback == b tests a > b
    out = %R independence_test($y ~ feedback == "$b",\
                               data=filter(rdf, feedback %in% c("$a", "$b")),\
                               alternative="$alternative")
    out = str(out)
    match = re.search('Z = (.*), p-value = (.*)', out)
    try:
        z, p = map(float, match.groups())
    except:
        print('Cannot parse independence test')
        print(out)
        raise Exception()
    
    x = rdf.set_index('feedback')[y]
    d = cohen_d(x.loc[a], x.loc[b])
    write_tex(f'independence-{y}-{a}-{b}', f'$d = ${d:.2f}, Z = {z:.2f}, {pval(p)}$')

rdf = pdf[['test', 'feedback']].rename(columns={
    'test': 'score'
})

In [ ]:
%%R -i rdf
library(dplyr)
library(coin)

In [ ]:
@figure()
def plot_test():
    if EXPERIMENT == 6:
        plt.figure(figsize=(12,4))
    
    sns.swarmplot('feedback', 'test', data=pdf, palette=palette, alpha=0.5, order=fb_order)
    sns.pointplot('feedback', 'test', data=pdf, palette=palette, order=fb_order, 
                  scale=1, capsize=0.1, markers='o')
    plt.xlabel('Feedback')
    plt.ylabel(nice_names['score'])
    test = 'Test' if EXPERIMENT == 1 else 'Transfer'
    reformat_labels()
#     if EXPERIMENT == 4:
#         plt.axhline(8.879, color='black')

In [ ]:
write_kruskal('score')

In [ ]:
if EXPERIMENT == 6:
    write_independence_test('score', 'both', 'none')
    write_independence_test('score', 'info_only', 'none')
    write_independence_test('score', 'reward_only', 'none')
    write_independence_test('score', 'both', 'reward_only')
    write_independence_test('score', 'both', 'info_only')
else:
    write_independence_test('score', 'meta', 'none')
    if 'action' in fb_order:
        write_independence_test('score', 'meta', 'action')
        write_independence_test('score', 'action', 'none')

In [ ]:
N_BOOT = 10000
test_score = pdf.set_index('feedback').test

def ci(xs):
    return np.quantile(xs, [0.025, 0.975])

def boot_means(n=N_BOOT):
    r = {}
    for fb in fb_order:
        x = test_score.loc[fb]
        means = [x.sample(frac=1, replace=True).mean() for _ in range(n)]
        r[fb] = np.array(means)
    return r

In [ ]:
bm = boot_means()
for fb in fb_order:
    x = test_score.loc[fb].mean()
    a, b = ci(bm[fb])
    if fb == 'none':
        tex = (rf'${x:.2f}$ points/trial (95\% CI: [${a:.2f}$, ${b:.2f}$])')
    else:
        tex = (rf'${x:.2f}$ points/trial; 95\% CI: [${a:.2f}$, ${b:.2f}$]')
    write_tex(f'mean-{fb}', tex)

# Learning curves

In [ ]:
def learning_curve(var):
    df = mdf.copy()
    df.trial_index += 1
    sns.lineplot('trial_index', var, hue='feedback', 
                 data=df, hue_order=fb_order, palette=palette)
    plt.ylabel(nice_names[var])
    plt.xlabel('Trial Number')
    plt.gca().grid(axis='x')
    split = mdf.query('block == "training"').trial_index.max()
    plt.axvline(split+1.5, c='k', ls='--', alpha=0.3)
    plt.xticks([1, *range(5, 31, 5)])
    plt.xlim(df.trial_index.min()-0.5, df.trial_index.max()+0.5)
    reformat_legend()
    
figure(var='backward')(learning_curve)

In [ ]:
figure(var='score')(learning_curve)

# Process

## Summary in test block

In [ ]:
def report_ratio(df, key):
    name = key.replace('_', '-')
    X = df.groupby(['feedback', key]).apply(len)
    rate = 100 * df.groupby('feedback')[key].mean()

    for c in fb_order:
        r = rate[c]
        write_tex(f'{name}-{c}-percent', f"${r:.1f}$\%")


report_ratio(mdf.query('block == "test"').copy(), 'backward')

## Mediation

In [ ]:
if EXPERIMENT in (4, 6):
    print("Mediation not implemented for this analysis")
    exit()

In [ ]:
rdf = mdf.query('block == "test"').copy().rename(columns={'information': 'info'})
# rdf['feedback'] = (rdf.feedback != 'none').astype(int)

if EXPERIMENT == 6:
    factors = ['info', 'reward']
    for c in factors:
        rdf[c] = rdf[c].astype(float)
else:
    factors = ['action', 'meta']
    fb = rdf.pop('feedback')
    for c in factors:
        rdf[c] = (fb == c).astype(int)

rdf['clicked'] = rdf.clicked.astype(int)
rdf = rdf[[*factors, 'backward', 'clicked', 'n_clicks', 'score', 'trial_index', 'stim_i']].reset_index()
rdf.backward.fillna(0, inplace=True)
# rdf.trial_index = rdf.trial_index.astype(float)
# rdf.query('action == 0', inplace=True)
rdf = rdf.groupby('pid').mean()
rdf.trial_index -= rdf.trial_index.min()

In [ ]:
%%R -i rdf -o score
fit = lm(score ~ backward, data=rdf)
score = summary(fit)
score

In [ ]:
%%R -i rdf -o score
fit = lm(score ~ backward, data=rdf)
score = summary(fit)
score

In [ ]:
score = dict(score.items())
df = score['df'][1]
est, std, t, p = score['coefficients'][1]
write_tex(f'score-backward', f'${est:.2f}$')
write_tex(f'score-backward-test', f'$t({df}) = {t:.2f}$, ${pval(p)}$')

In [ ]:
%%R
local({r <- getOption("repos")
       r["CRAN"] <- "https://cloud.r-project.org" 
       options(repos=r)
})
# install.packages('mediation')

In [ ]:
%%R -i rdf -i EXPERIMENT -o back -o med_out
library(mediation)

if ('action' %in% colnames(rdf)) {
    back_fit = lm(backward ~ action + meta, data=rdf)
    score_fit = lm(score ~ backward + action + meta, data=rdf)
# } else if (EXPERIMENT == 6) {
#     back_fit = lm(backward ~ info + reward, data=rdf)
#     score_fit = lm(score ~ backward + info + reward)
} else {
    back_fit = lm(backward ~ meta, data=rdf)
    score_fit = lm(score ~ backward + meta, data=rdf)
}
med_out = mediate(back_fit, score_fit, treat="meta", mediator="backward")
back = summary(back_fit)
summary(med_out)

In [ ]:
back = dict(back.items())
df = back['df'][1]
for i, name in enumerate(fb_order[1:], start=1):
    est, std, t, p = back['coefficients'][i]
    write_tex(f'backward-lm-{name}', f'$t({df}) = {t:.3f}$, ${pval(p)}$')

In [ ]:
names = {
    'acme': 'd0',
    'ade': 'z0',
    'total': 'tau',
    'prop': 'n0',
}
med = dict(med_out.items())

for k, v in names.items():
    est = med[v + ('.coef' if v == 'tau' else '')][0]
    lo, hi = med[v+'.ci']
    p = med[v+'.p'][0]
    write_tex(f'mediation-{k}', f'{est:.3f}')
    write_tex(f'mediation-{k}-ci', f'95\% CI: [{lo:.3f}, {hi:.3f}], ${pval(p)}$')

In [ ]:
print("Success!")